In [15]:
from dotenv import load_dotenv
import os

ENV_PATH = '.env'

confluence_user_id = 11266


if not os.path.exists(ENV_PATH):
    raise Exception('create dot env file with cookies and request_verification_token')

load_dotenv(ENV_PATH)

## monitor network traffic and copy paste out of as request
cookie = os.environ['cookie']
request_verifcation_token = os.environ['request_verifcation_token'] ## pulled from people API


In [16]:
def generate_headers(cookie, request_verifcation_token :str =  None ):

    headers =  {
        'authority': 'centerstage.sie.sony.com',
        'accept': 'application/json, text/plain, */*',
        'cookie': cookie,
    }

    if request_verifcation_token: headers.update({    '__requestverificationtoken': request_verifcation_token})

    return headers

# transport

In [17]:
from aiohttp import ClientSession

class GetData_Error(Exception):
  def __init__(self, status, message):

    super(f'get_data {status} - { message }')
    

async def get_data(
    url: str,
    headers,
    json = None,
    session: ClientSession = None,
    method :str = 'GET',
    verify_ssl : bool = False,
    return_raw: bool = False
) -> None:

    is_close = False

    if not session:
        session = ClientSession()
        is_close = True

    try:
      async with getattr(session, method.lower())(url = url, headers = headers, json = json,  verify_ssl = verify_ssl) as res:
          
          if return_raw:
            return res

          if not res.status == 200:
            raise GetData_Error(status = res.status, message = await res.text())

          return await res.json()
    
    finally:
      if is_close:
        await session.close()


async def get_data_stream(
    url: str,
    headers,
    method :str = 'GET',
    verify_ssl : bool = False
) -> None:

    content = bytearray()
    
    async with ClientSession() as session:
      async with session.get(url = url, headers = headers, verify_ssl = verify_ssl) as res:
        
        if not res.status == 200:
          raise GetData_Error(status = res.status, message = await res.text())

        async for chunk in res.content:
          content += chunk

        return content


# Routes

In [18]:
from pprint import pprint
import aiohttp

async def get_people(user_id,
                     cookie,
                     request_verifcation_token : str = None, # optional
                     debug_api: bool = False,
                     return_raw : bool = False,
                     session : aiohttp.ClientSession = None
                     ):
                     
    url = f"https://centerstage.sie.sony.com/api/people/{user_id}"

    headers = generate_headers(cookie, request_verifcation_token)

    if debug_api:
        pprint({'url': url, 'headers': headers
                })

    return await get_data(method="GET", url=url, headers=headers, return_raw= return_raw, session = session)


await get_people(user_id=confluence_user_id,
                 cookie=cookie,
                 debug_api=False,
                 return_raw = False
                 )


{'Id': 11266,
 'FirstName': 'Jim',
 'LastName': 'Ryan',
 'Pronouns': None,
 'FullName': 'Jim Ryan',
 'JobTitle': 'President & CEO',
 'PrimaryDepartment': 'Sony Interactive Entertainment - Global',
 'PrimaryLocation': 'GBR-London-10GMS',
 'PrimaryCompany': 'Sony Interactive Entertainment Europe Ltd.',
 'WorkPhone': '+44 20 7859 5500',
 'Mobile': '07802-227-021',
 'Extension': 'False',
 'Email': 'Jim.Ryan@sony.com',
 'IsFollowing': False,
 'Followers': 0,
 'Following': 0,
 'InfluenceScore': 17,
 'IsMe': False,
 'AssetId': 405406,
 'ImageUrl': '/api/asset/405406',
 'ExpertiseUrl': '/api/people/11266/expertise',
 'InterestsUrl': '/api/people/11266/interests',
 'FollowersUrl': '/api/people/11266/followers',
 'AdditionalFields': {'Find Me': 'https://playstation.service-now.com/x_nuvo_eam_fm_view_v2.do?app=user#?s=97c37c7b1b393410efb697d58d4bcbe3&search=b969b41b1bbb63001f64bb7acd4bcb80',
  'OrgChart': 'https://people.sie.sony.com?user=jim.ryan@sony.com',
  'costcenter': 'GBT1060052',
  'Expor

In [19]:
class DownloadAvatar_InvalidAssetId(Exception):
    def __init__(self, user_id = None, asset_id = None):
        if user_id :
            message = f"unable to download avatar for {user_id}"
        
        if not asset_id and not user_id:
            message = f"unable to download avatar.  provide a valid asset_id"

        super().__init__(message)


async def download_avatar(asset_id,
                    cookie,
                    size=5,
                    request_verifcation_token: str = None,  # optional
                    debug_api: bool = False):

    url = f'https://centerstage.sie.sony.com/api/asset/{asset_id}/type/Person?size={size}'

    headers = generate_headers(cookie, request_verifcation_token)
    headers.update({'accept' : 'image/jpeg'})

    if debug_api:
        pprint({'url': url, 'headers': headers
                })

    return await get_data_stream(method="GET", url=url, headers=headers)

await download_avatar(asset_id=395934,
                 cookie=cookie,
                 debug_api=False)



bytearray(b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x00\x00\x00\x01\x00\x08\x06\x00\x00\x00\\r\xa8f\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x00\x04gAMA\x00\x00\xb1\x8f\x0b\xfca\x05\x00\x00\x00\tpHYs\x00\x00\x0e\xc3\x00\x00\x0e\xc3\x01\xc7o\xa8d\x00\x00\xff\xa5IDATx^\xac\xdd\xe5\xb7-\xcbU\xfe\xf1\xfc\x1b\xfc\xc6\x80\xc1\x80\x10!\x9e\xdc\xd8\x8d\xbb\x93\x00\t\x16\x82n 8\x1c\xdc\x83&\xb8\xbb\xbb\xbb\xbb\xbb\xbb{\x8cd\xdf\x04B\x02c\xc0\x8b\xf5[\x9f\xce\xfd\xee\xcc]\xa7\xd7>\xe7\x02/\xe6\xa8\xea\xea\xea\xaaYs\xce\xe7\xa9Y\xddk\x9fs\xa7\x9f\xf9\x99\x9f\xf9\x9b_\xf8\x85_8?\x96\xe7?\xfd\xd3?\xbd\x95?\xfb\xb3?{\xfes?\xf7s\x17\xe5\xcf\xff\xfc\xcf\x9f\x14\xcf&\xbf\xf4K\xbft\xfe\xcb\xbf\xfc\xcb[\xf9+\xbf\xf2+\x97\xe4W\x7f\xf5W/\xe4\xd7\x7f\xfd\xd77\xf9\x8d\xdf\xf8\x8dM~\xf37\x7f\xf3\x92\xfc\xd6o\xfd\xd6\xf9o\xff\xf6o_\'\xbf\xf3;\xbfs\xfe\xbb\xbf\xfb\xbb\xe7\xbf\xf7{\xbfw!\xf3\xfa\xf7\x7f\xff\xf7\xcf\xff\xe0\x0f\xfe`+\xf55\xf6\x8f\xfe\xe8\x8f\x9e\x7f\xed\xd7~\xed\xf9\x0b_\xf8\xc2\xf

In [20]:
class GetTeam_RequestVerificationToken_Error(Exception):
    def __init__(self, user_id):
        super().__init__(f"unable to retrieve team for {user_id} - API requires `request_verification_token`")


async def get_team(user_id,
                   cookie,
                   request_verifcation_token,
                   return_raw: bool = False,
                   debug_api: bool = False,
                   session: aiohttp.ClientSession = None
                   ):
    
    if not request_verifcation_token:
        raise GetTeam_RequestVerificationToken_Error(user_id)

    url = f'https://centerstage.sie.sony.com/api/people'

    body = {"managerId": user_id, "limit": 100}

    headers = generate_headers(cookie, request_verifcation_token)

    headers = {**headers,
               'referer': f'https://centerstage.sie.sony.com/person/{user_id}'}

    if debug_api:
        pprint({'url': url, 'body': body,
                'headers': headers
                })

    data = await get_data(method="POST", url=url, headers=headers, json=body, verify_ssl=False, session = session)

    if return_raw:
        return data
    
    user_ls = data.get('Results')

    return user_ls or []

await get_team(user_id=669,
               cookie=cookie,
               request_verifcation_token=request_verifcation_token,
               debug_api=False,
               return_raw=False,
            
               )


[{'Id': 763,
  'FullName': 'Andrew Lipscomb',
  'LastName': 'Lipscomb',
  'FirstName': 'Andrew',
  'IsMe': False,
  'AssetId': 412767,
  'Email': 'Andrew.Lipscomb@sony.com',
  'JobDescription': '',
  'Manager': 'William Budnovitch',
  'PrimaryDepartment': 'PDS - Analytics Engineering (William Budnovitch)',
  'PrimaryCompany': 'Sony Interactive Entertainment',
  'PrimaryLocation': 'USA-ME-Telecommute',
  'Extension': 'False',
  'WorkPhone': '650-295-6534',
  'Mobile': None,
  'JobTitle': 'Manager, Software Engineering',
  'ImageUrl': '/api/asset/412767',
  'ExpertiseUrl': '/api/people/763/expertise',
  'InterestsUrl': '/api/people/763/interests',
  'FollowersUrl': '/api/people/763/followers',
  'HasTwoFactorSecret': False,
  'QueryStringUtmParameters': 'utm_source=interact&utm_medium=person_search'},
 {'Id': 3168,
  'FullName': 'Ashley Galazia',
  'LastName': 'Galazia',
  'FirstName': 'Ashley',
  'IsMe': False,
  'AssetId': 396095,
  'Email': 'Ashley.Galazia@sony.com',
  'JobDescription

# Class
- using classes facilitates recursive instatiation

In [21]:
from dataclasses import dataclass, field
import asyncio
import os

# async def gather_with_concurrency(n, *coros):
#     semaphore = asyncio.Semaphore(n)

#     async def sem_coro(coro):
#         async with semaphore:
#             return await coro
#     return await asyncio.gather(*(sem_coro(c) for c in coros))


@dataclass
class CCPerson:
    id: int
    asset_id: int
    first_name: str
    last_name: str
    job_title: str
    phone: str
    email: str
    department: str
    location: str

    cookie: str = field(repr=False)
    request_verifcation_token: str = field(repr=False)

    manager: dict = None
    reports_ls: list = None


    @classmethod
    async def _from_json(cls, ccobj, cookie, request_verifcation_token, session = None):

        ccp = cls(
            id=ccobj['Id'],
            asset_id=ccobj['AssetId'],
            first_name=ccobj['FirstName'],
            last_name=ccobj['LastName'],
            job_title=ccobj['JobTitle'],
            phone=ccobj.get('WorkPhone') or ccobj.get('Mobile'),
            email=ccobj['Email'],
            department=ccobj['PrimaryDepartment'],
            location=ccobj['PrimaryLocation'],
            cookie=cookie,
            request_verifcation_token=request_verifcation_token,
        )

        # ccp.manager = await cls.get_from_id(user_id = ccobj['ManagerId'], cookie=cookie,
        #                     request_verifcation_token=request_verifcation_token) if ccobj['ManagerId'] != ccobj['Id'] else None

        await ccp.get_team(session = session)

        return ccp

    @classmethod
    async def get_from_id(cls, user_id,
                          cookie, request_verifcation_token,
                          debug_prn: bool = False, debug_api: bool = False,
                          return_raw: bool = False,
                          session: aiohttp.ClientSession = None):

        if debug_prn:
            print(f'retrieving {user_id}')

        ccobj = await get_people(user_id=user_id, cookie=cookie,
                                 request_verifcation_token=request_verifcation_token, debug_api=debug_api,
                                 session = session
                                 )

        if return_raw:
            return ccobj

        return await cls._from_json(ccobj=ccobj, cookie=cookie, request_verifcation_token=request_verifcation_token, session = session)

    async def get_team(self, return_raw: bool = False, debug_api: bool = False, session : aiohttp.ClientSession = None):

        user_ls = await get_team(user_id=self.id,
                                 cookie=self.cookie,
                                 request_verifcation_token=self.request_verifcation_token,
                                 #  return_raw = return_raw,
                                 debug_api=debug_api,
                                 session = session)

        await asyncio.sleep(5)

        if return_raw:
            return user_ls

        self.reports_ls = await asyncio.gather(*[CCPerson._from_json(ccobj=ccobj, cookie=cookie, request_verifcation_token=request_verifcation_token, session = session) for ccobj in user_ls])

    async def download_avatar(self, folder_path='./images', img_name=None, debug_api: bool = False, return_raw: bool = False):
        if not self.asset_id:
            raise DownloadAvatar_InvalidAssetId(user_id=self.id)

        stream_content = await download_avatar(asset_id=self.asset_id,
                                               cookie=cookie,
                                               debug_api=debug_api)

        if return_raw:
            return stream_content

        if not os.path.exists(folder_path):
            os.mkdir(folder_path)

        img_name = f"{img_name.replace('.png', '') or self.id}.png"

        with open(f"{folder_path}/{img_name}", "wb") as f:
            f.write(stream_content)

        return True


In [22]:
test = await CCPerson.get_from_id(user_id=669,
                     cookie=cookie,
                     request_verifcation_token = request_verifcation_token,
                     return_raw=False)


await test.download_avatar(return_raw = False, img_name = test.email)

True

In [23]:

def print_hierarchy(ccperson , level = 1):
    print( f'{">"*level } {ccperson.id} - {ccperson.first_name} {ccperson.last_name}')

    if not ccperson.reports_ls:
        return
    
    for report in ccperson.reports_ls:
        print_hierarchy(report, level+1)

print_hierarchy(test)


> 669 - William Budnovitch
>> 763 - Andrew Lipscomb
>>> 35972 - Ahmed Raza
>>> 13319 - Anirudh Nirmal Rajanala
>>> 42338 - Michael Marquez
>> 3168 - Ashley Galazia
>>> 36098 - Michelle DePonte
>> 762 - Blake Ellison
>>> 1454 - Brian Mallis
>>>> 37236 - Jesus Gonzalez
>>> 604 - David Humphrey
>>> 32951 - Marc Pallister
>>> 7120 - Mauro Braghieri
>>> 40940 - Mauro Braghieri
>>>> 35740 - Rahul Govind
>>> 3408 - William Honaker
>>>> 27733 - James Mitchell
>>>> 43028 - Wesley Limtiaco
>> 15021 - John Taft
>>> 13799 - Emma Pusey
>>>> 15501 - Andy Christer
>>>> 19715 - Anna Banas
>>>> 12804 - Antony Rowe
>>>> 7169 - Javnish Parikh
>>> 6964 - Filippo Focardi
>>>> 6742 - Andrew Male
>>>> 732 - Saroja Velamati
>>>>> 947 - Anup Marka
>>>>> 4102 - Jones Nakka
>>>> 19253 - Susana Martin Gutierrez
>>> 32220 - Jae Myong Wilson
>>> 30602 - Nate Bear
>>>> 22472 - Anna Favis
>>>> 22457 - Brook Girma
>>>> 35334 - Catherine Choe
>>>> 23980 - Omprakash Basant Gurubaxani
>>>> 20298 - Thomas Murphey
>>> 3508

In [24]:
# import domolibrary.client.DomoAuth as dmda
# import domolibrary.classes.DomoUser as dmdu

# auth = dmda.DomoFullAuth(domo_username=os.environ['DOMO_USERNAME'],
#                   domo_password=os.environ['DOMO_PASSWORD'],
#                   domo_instance='playstation-beta'
#                   )

# await auth.get_auth_token()

# domo_user_id = 1360345502

# domo_user = await dmdu.DomoUser.get_by_id(user_id = domo_user_id, auth = auth)

# await domo_user.download_avatar(folder_path = './images/', img_name=domo_user.email_address)


In [25]:
avatars_to_get = ['anna.banas@sony.com',
 'jace.mclean@domo.com',
 'geanrcarlo.palavicini@sony.com',
 'benjamin.audy@sony.com',
 'thomas.murphey@sony.com',
 'bal.sanghera@sony.com',
 'beth.nicholson@sony.com!!',
 'brook.girma@sony.com',
 'test_pdp3@sony.com',
 'test_pdp1@sony.com',
 'test1@sony.com',
 'catherine.choe@sony.com',
 'annacarissa.favis@sony.com',
 'system@domo.com',
 'jacob.peterson@domo.com',
 'karthik.prasad@sony.com',
 'omprakashbasant.gurubaxani@sony.com',
 'sean.white1@sony.com',
 'leah.welch@sony.com',
 'ed.dawson@sony.com!!',
 'william.honaker@sony.com',
 'test_pdp2@sony.com',
 'sie.domo.admin@sony.com',
 'brent.curry@sony.com',
 'sie.domo.reset@sony.com',
 'dhinesh.baulraj@sony.com!!',
 'bib@domo.com']

avatars_to_get[0:5]

['anna.banas@sony.com',
 'jace.mclean@domo.com',
 'geanrcarlo.palavicini@sony.com',
 'benjamin.audy@sony.com',
 'thomas.murphey@sony.com']

In [26]:
import aiohttp
connector = aiohttp.TCPConnector(limit=60)
session = aiohttp.ClientSession(connector=connector)

test = await CCPerson.get_from_id(user_id=confluence_user_id,
                     cookie=cookie,
                     request_verifcation_token = request_verifcation_token,
                     return_raw=False,
                     session = session
                     )


test

ClientOSError: [WinError 10054] An existing connection was forcibly closed by the remote host